In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
def splitDate(date):
    date = str(date)
    Y = int(date[:4])
    M = int(date[4:6])
    D = int(date[6:])
    return (Y, M, D)

In [3]:
def getWeek(date):
    return datetime(*date).weekday()

In [4]:
read_csv = pd.read_csv("/Users/jooh8/Documents/GitHub/Quant/data/A000660/Index_A000660.txt", sep=" ", low_memory=False)

In [5]:
read_csv["DoW"] = read_csv["날짜"].apply(lambda x: getWeek(splitDate(x)))

In [6]:
weekList = []
preW = 0
week = 0

for W in read_csv["DoW"]:
    if W < preW:
        week += 1
    weekList.append(week)
    preW = W

In [7]:
read_csv["WEEK"] = weekList

In [8]:
for period in range(10):
    
    read_csv["고가_{}".format(period)] = read_csv["고가"].shift(periods=period).fillna(0)

In [9]:
for period in range(10):
    
    read_csv["최고가_{}".format(period)] = read_csv[["고가_{}".format(i) for i in range(period + 1)]].apply(np.max, 1)

In [10]:
for period in range(10):
    
    read_csv["Target_{}".format(period)] = read_csv["시가"] * 1.05 < read_csv["고가_{}".format(period)]

In [11]:
columns = list(read_csv.columns)

In [12]:
columns

['날짜',
 '시가',
 '고가',
 '저가',
 '종가',
 '거래량',
 '거래대금',
 '상장주식수',
 '시가총액',
 '외국인현보유수량',
 '기관순매수',
 'A Ratio_20',
 'B Ratio_20',
 'Aroon_20 Up',
 'Down',
 'Aroon Osillator_20',
 'ATR_14',
 'BB-RSI_종가_10',
 'BB-RSI_20_2.00 상한',
 'BB-RSI 하한',
 'BPDL RSI_14',
 'BPDL Stochatic_14',
 'Chande Momentum Oscillator_10',
 "Chaikin's Volatility_14,14",
 'CompuTrac Volatility_10',
 '+DI_14',
 '-DI_14',
 'ADX_14',
 'ADXR_14',
 '+DI(simple)_14',
 '-DI(simple)_14',
 'ADX(simple)_14,14',
 'ADXR(simple)_14,14,14',
 'Energy+_14',
 'Energy-_14',
 'High Low Envelope_1',
 'High Low Oscillator_3',
 'Inertia_20,14,10',
 'Klinger Oscillator',
 'Linear Trend Oscillator_10,20',
 'LRS_종가,14',
 'Mass Index_20,9',
 'Morris Mixed Momentum',
 'New BPDL HiLo Index MA_14,7',
 'Open Difference_15',
 'QStick_14',
 'Random Walk Index_15,3',
 'RCI_5',
 'RCI_9',
 'RCI_13',
 'RCI_18',
 'Relative Volatility Index_단순,14,10',
 'Reverse 단기_12',
 ' 장기_24',
 'RSI_종가,14',
 'RSI(simple)_종가,14',
 'Sigma_종가,20',
 'SMI_5,3,3',
 'Standard D

In [13]:
momentums = columns[11:-32]
print(momentums)

['A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down', 'Aroon Osillator_20', 'ATR_14', 'BB-RSI_종가_10', 'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14', 'Chande Momentum Oscillator_10', "Chaikin's Volatility_14,14", 'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14', '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14', 'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14', 'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10', 'Klinger Oscillator', 'Linear Trend Oscillator_10,20', 'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum', 'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14', 'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18', 'Relative Volatility Index_단순,14,10', 'Reverse 단기_12', ' 장기_24', 'RSI_종가,14', 'RSI(simple)_종가,14', 'Sigma_종가,20', 'SMI_5,3,3', 'Standard Deviation_14,2.00', 'Standard Error_종가,14', 'Fast %K_5', 'Fast %D_3', 'Slow %K_5,3', 'Slow %D_3', 'Slow(Simple) %K_5,3', 'Slow(Simple) %D_3', 

In [14]:
targets = columns[-10:]
print(targets)

['Target_0', 'Target_1', 'Target_2', 'Target_3', 'Target_4', 'Target_5', 'Target_6', 'Target_7', 'Target_8', 'Target_9']


In [15]:
X_train = read_csv[-2000:-100][momentums]
Y_train = read_csv[-2000:-100][targets]

In [16]:
Y_train.apply(np.sum, 0)

Target_0     32
Target_1     72
Target_2    189
Target_3    282
Target_4    345
Target_5    366
Target_6    402
Target_7    409
Target_8    438
Target_9    458
dtype: int64

In [17]:
X_test = read_csv[-100:][momentums]
Y_test = read_csv[-100:][targets]

In [18]:
Y_test.apply(np.sum, 0)

Target_0     2
Target_1     1
Target_2    10
Target_3    16
Target_4    23
Target_5    22
Target_6    22
Target_7    23
Target_8    21
Target_9    27
dtype: int64

In [19]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
import numpy as np

In [20]:
scaler = preprocessing.StandardScaler().fit(X_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [21]:
forest = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=5)

In [22]:
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [23]:
multi_target_forest.fit(scaler.transform(X_train), Y_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1)

In [24]:
predict = multi_target_forest.predict(scaler.transform(X_train))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [25]:
corr = predict == Y_train

In [26]:
corr.apply(np.mean, 0)

Target_0    0.994211
Target_1    0.971579
Target_2    0.941579
Target_3    0.931579
Target_4    0.935789
Target_5    0.935263
Target_6    0.940526
Target_7    0.947368
Target_8    0.952632
Target_9    0.953158
dtype: float64

In [27]:
np.mean(corr.apply(np.mean, 0))

0.9503684210526316

In [28]:
predict = multi_target_forest.predict(scaler.transform(X_test))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [29]:
corr = predict == Y_test

In [30]:
corr.apply(np.mean, 0)

Target_0    0.98
Target_1    0.99
Target_2    0.88
Target_3    0.96
Target_4    0.87
Target_5    0.91
Target_6    0.91
Target_7    0.95
Target_8    0.96
Target_9    0.96
dtype: float64

In [31]:
np.mean(corr.apply(np.mean, 0))

0.937

In [32]:
cm = confusion_matrix(np.array(Y_test["Target_9"]), predict[:, 9])

In [33]:
Sensitivity = cm[1, 1] / np.sum(cm[:, 1])
Accuracy = np.sum(cm.diagonal()) / np.sum(cm)

In [34]:
Sensitivity, Accuracy

(0.896551724137931, 0.96)

In [35]:
"""(0.8666666666666667, 0.95)"""

'(0.8666666666666667, 0.95)'